# Test

In [13]:
!{sys.executable} -m pip install --user -U kagglehub
!{sys.executable} -m pip install --user tflite-support
!{sys.executable} -m pip install --user tflite-runtime


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## **Step 0**
Installing kagglehub to get the google aiy model

In [14]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/aiy/tfLite/vision-classifier-food-v1")

print("Path to model files:", path)

Path to model files: /home/kristoffel/.cache/kagglehub/models/google/aiy/tfLite/vision-classifier-food-v1/1


## **Step 1**
Downloading the model

In [16]:
import os, kagglehub

# Download model (if not already cached)
food_dir = kagglehub.model_download("google/aiy/tfLite/vision-classifier-food-v1")
print("Model dir:", food_dir)
print("Contents:", os.listdir(food_dir))

# Helper to find the .tflite file
def find_tflite(base_dir):
    for root, _, files in os.walk(base_dir):
        for f in files:
            if f.lower().endswith(".tflite"):
                return os.path.join(root, f)
    raise FileNotFoundError("No .tflite file found")

model_path = find_tflite(food_dir)
print("Using model:", model_path)

# Helper to load labels if provided
def load_labels(base_dir):
    for root, _, files in os.walk(base_dir):
        for f in files:
            if f in ("labels.txt", "labelmap.txt", "labels.csv"):
                with open(os.path.join(root, f), "r", encoding="utf-8") as fh:
                    return [ln.strip() for ln in fh if ln.strip()]
    return None

labels = load_labels(food_dir)
print("Labels loaded:", labels is not None, f"(n={len(labels) if labels else 'N/A'})")


Model dir: /home/kristoffel/.cache/kagglehub/models/google/aiy/tfLite/vision-classifier-food-v1/1
Contents: ['1.tflite']
Using model: /home/kristoffel/.cache/kagglehub/models/google/aiy/tfLite/vision-classifier-food-v1/1/1.tflite
Labels loaded: False (n=N/A)


In [18]:
from tflite_runtime.interpreter import Interpreter
from tflite_support import metadata

model_path = "/home/kristoffel/.cache/kagglehub/models/google/aiy/tfLite/vision-classifier-food-v1/1/1.tflite"

# Load metadata
displayer = metadata.MetadataDisplayer.with_model_file(model_path)

print("Associated files:", displayer.get_packed_associated_file_list())

# If a labels file is embedded, you can extract it:
for fname in displayer.get_packed_associated_file_list():
    if "label" in fname.lower():
        labels = displayer.get_associated_file_buffer(fname).decode("utf-8").splitlines()
        print("Found labels:", labels[:10], "... total:", len(labels))


ImportError: libusb-1.0.so.0: cannot open shared object file: No such file or directory

## **Step 2**
Locate the model and labels

In [19]:
import os, numpy as np
from PIL import Image
import tflite_runtime.interpreter as tflite

# Point this to the path you printed from kagglehub:
food_dir = "/home/kristoffel/.cache/kagglehub/models/google/aiy/tfLite/vision-classifier-food-v1/1"
image_path = "Banana-Single.jpg"  # <- change to your test image

def find_tflite(base_dir):
    for root, _, files in os.walk(base_dir):
        for f in files:
            if f.lower().endswith(".tflite"):
                return os.path.join(root, f)
    raise FileNotFoundError("No .tflite found under " + base_dir)

def load_labels(base_dir):
    # common label file names
    candidates = ("labels.txt", "labelmap.txt", "labels.csv")
    for name in candidates:
        p = os.path.join(base_dir, name)
        if os.path.exists(p):
            with open(p, "r", encoding="utf-8") as f:
                return [ln.strip() for ln in f if ln.strip()]
    # also check subfolders
    for root, _, files in os.walk(base_dir):
        for f in files:
            if f in candidates:
                with open(os.path.join(root, f), "r", encoding="utf-8") as fh:
                    return [ln.strip() for ln in fh if ln.strip()]
    return None

model_path = find_tflite(food_dir)
labels = load_labels(food_dir)
print("Using model:", model_path)
print("Labels loaded:", labels is not None, f"(n={len(labels) if labels else 'N/A'})")

# Init interpreter
interpreter = tflite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
inp  = interpreter.get_input_details()[0]
outp = interpreter.get_output_details()[0]
print("Input details:", inp)
print("Output details:", outp)

# Prepare image with correct size & dtype
H, W = int(inp["shape"][1]), int(inp["shape"][2])
img = Image.open(image_path).convert("RGB").resize((W, H))

if inp["dtype"] == np.uint8:
    x = np.expand_dims(np.array(img, dtype=np.uint8), 0)     # quantized path
else:
    x = np.expand_dims(np.array(img, dtype=np.float32)/255., 0)  # float path

# Inference
interpreter.set_tensor(inp["index"], x)
interpreter.invoke()
y = interpreter.get_tensor(outp["index"])[0]

# Dequantize if needed
if outp["dtype"] == np.uint8:
    scale, zero = outp["quantization"]
    probs = (y.astype(np.float32) - zero) * scale
else:
    probs = y.astype(np.float32)

# Top-5
top5 = probs.argsort()[-5:][::-1]
print("\nTop-5 predictions:")
for i in top5:
    name = labels[i] if labels and i < len(labels) else f"class_{i}"
    print(f"{name:30s}  {probs[i]:.3f}")


Using model: /home/kristoffel/.cache/kagglehub/models/google/aiy/tfLite/vision-classifier-food-v1/1/1.tflite
Labels loaded: False (n=N/A)
Input details: {'name': 'input', 'index': 0, 'shape': array([  1, 192, 192,   3], dtype=int32), 'shape_signature': array([  1, 192, 192,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.0078125, 128), 'quantization_parameters': {'scales': array([0.0078125], dtype=float32), 'zero_points': array([128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
Output details: {'name': 'MobilenetV1/Predictions/Softmax', 'index': 90, 'shape': array([   1, 2024], dtype=int32), 'shape_signature': array([   1, 2024], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

Top-5 predictions:
class_1008                      0.164
class_315 